In [1]:
import pandas as pd  
from Functions_RestoreEvaluation import *

In [2]:
# Initialize the circuit and its equivalent graph
# Remove the service buses ( with 's' and 'x' prefix in bus name)--- This to get the primary circuit grap

DSSCktobj,G_init,conv_flag = initialize() #initial circuit set up

G_primary = G_init.copy()

for e in G_init.edges():
    (u,v) = e
    if u[0] == 's' or u[0] == 'x' or v[0] == 's' or v[0] == 'x':
        G_primary.remove_edge(u, v)
        
G_primary.remove_nodes_from(list(nx.isolates(G_primary)))

In [3]:
# Harshal's: finding the buses with missing co-ordinates
def find_missing_elements(G_primary, txt_file_path):
    # Create an excel file containing current nodes
    nodes_list = list(G_primary.nodes)
    G_primary_nodes = pd.DataFrame(nodes_list, columns=['G_primary_nodes'])
    G_primary_nodes.to_excel('G_primary_nodes.xlsx', index = False)

    # Read Excel file
    excel_file_path = 'G_primary_nodes.xlsx'
    df = pd.read_excel(excel_file_path)
    [row, col] = df.shape

    # Read text file (lower letter)
    with open(txt_file_path, 'r') as txt_file:
        txt_data = [element.lower().strip() for row in txt_file.read().split(',') for element in row.splitlines()]

    missing_elements = []
    for r in range(row):
        string = df['G_primary_nodes'][r].replace("'", '')
        string = string.replace(" ", "")
        if string not in txt_data:
            missing_elements.append(string)

    # save missing_elements as an excel file
    missing_df = pd.DataFrame(missing_elements, columns = ['Missing Elements'])
    missing_df.to_excel('missing_elements.xlsx', index = False)
    
    return missing_elements

In [4]:
missing_elements = find_missing_elements(G_primary, 'Buscoords.txt') # get all the buses with missing coordinates
len(missing_elements)

67

In [5]:

#Harshal's:  Reducing graph by removing buses with missing coordinates
for i in range(len(missing_elements)):
    if len(list(G_primary.edges(missing_elements[i]))) >= 2:
        G_primary.add_edge(list(G_primary.edges(missing_elements[i]))[0][1], list(G_primary.edges(missing_elements[i]))[1][1])
        G_primary.remove_node(missing_elements[i])

    if len(list(G_primary.edges(missing_elements[i]))) == 1:
        G_primary.remove_node(missing_elements[i])

In [6]:
Gdir=nx.DiGraph()
edge_list= list(G_init.edges()) # Converting the original graph(both primary and secondary) for finding upstream nodes
Gdir.add_edges_from(edge_list)

In [7]:
Load_Buses={} # Initialize a dictionary which will assign a set of loads connected to buses in G_primary
i= DSSCktobj.dss.Loads.First()
while i>0:
    elemName =  DSSCktobj.dss.CktElement.Name()
    bus_connectn = DSSCktobj.dss.CktElement.BusNames()[0].split('.')[0]
    map_bus  =  bus_connectn
    while len(map_bus)!=0:
        map_bus = list(Gdir.predecessors(map_bus))[0]
        if map_bus in G_primary.nodes():
            break
    Load_Buses[elemName] = map_bus #associated primary bus for each load
    i = DSSCktobj.dss.Loads.Next()
        

In [8]:
load_buses = np.array(list(Load_Buses.values())) #all the load buses
load_elems = list(Load_Buses.keys()) #all the load elements
Bus_PDem={} #Active power demand at bus
Bus_QDem={} # Reactive power demand at bus

In [9]:
#----- Acquiring the connected active power and reactive power load at all the buses (nodes)
nodelist = G_primary.nodes()
for n in nodelist:
    Pld_tot= 0 
    Qld_tot = 0
    load_names=[load_elems[y] for y in np.where(load_buses == n)[0]]
    for ld in load_names:
        DSSCktobj.dss.Circuit.SetActiveElement(ld)
        Pld_tot += float(DSSCktobj.dss.Properties.Value('Kw'))
        Qld_tot += float(DSSCktobj.dss.Properties.Value('Kvar'))
    Bus_PDem[n] = Pld_tot
    Bus_QDem[n] = Qld_tot

In [10]:
#---- Adding the total active and reactive power demand of loads as the node attributes in the graph
nx.set_node_attributes(G_primary, Bus_PDem, name="active power")
nx.set_node_attributes(G_primary, Bus_QDem, name="reactive power")

In [11]:
len(G_primary.nodes())

2454

In [12]:
G_primary.nodes(data=True) # You can see that the active and reactive power has been attributed to each of the nodes in the primary graph

NodeDataView({'m1009763': {'active power': 0, 'reactive power': 0}, 'l2673322': {'active power': 3.89, 'reactive power': 0.974925898734993}, 'm1069148': {'active power': 0, 'reactive power': 0}, 'l2673309': {'active power': 10.17, 'reactive power': 2.54884225967478}, 'm1069588': {'active power': 0, 'reactive power': 0}, 'l2804270': {'active power': 5.84, 'reactive power': 1.46364196622426}, 'm3036164': {'active power': 0, 'reactive power': 0}, 'l2935553': {'active power': 9.73, 'reactive power': 2.43856786495925}, 'm1209749': {'active power': 0, 'reactive power': 0}, 'l2748840': {'active power': 5.84, 'reactive power': 1.46364196622426}, 'm1108264': {'active power': 0, 'reactive power': 0}, 'm1108263': {'active power': 0, 'reactive power': 0}, 'l3085398': {'active power': 9.73, 'reactive power': 2.43856786495925}, 'm1026891': {'active power': 0, 'reactive power': 0}, 'l3216367': {'active power': 9.73, 'reactive power': 2.43856786495925}, 'm1069468': {'active power': 0, 'reactive power'

In [13]:
node_df = pd.DataFrame.from_dict(dict(G_primary.nodes(data=True)), orient='index')

In [14]:
node_df_desc =  node_df.sort_values(by=['reactive power'], ascending=False)
node_df_desc.head(20)

,active power,reactive power
l3234149,287.64,72.089379
l2767340,86.30,21.628819
l3207907,67.26,16.856945
l2803199,57.53,14.418377
l2915542,57.53,14.418377
l3215203,57.53,14.418377
l3047289,57.53,14.418377
l3011293,57.53,14.418377
l2814529,57.53,14.418377
l3215385,48.01,12.032440


The final graph to be used in the restoration work is: G_primary

In [15]:
list(Gdir.successors('_hvmv_sub_lsb'))
     #I do this till I find a node in G_primary that is at the head
list(Gdir.successors('d5710794-3_int'))

['e192860']

In [16]:
substation_id  = 'e192860'

Now evaluating the load restored by repairing the outage line

In [17]:
Gdir.out_edges('m1209822') #1: 37215

OutEdgeDataView([('m1209822', 'm1209823'), ('m1209822', '226-23745'), ('m1209822', 'l2992556')])

In [18]:
Gdir.in_edges('l2936216') #2: 23214

InEdgeDataView([('m1209819', 'l2936216'), ('226-23745', 'l2936216')])

In [19]:
Gdir.in_edges('l2823611') #3: 8433

InEdgeDataView([('m1209787', 'l2823611')])

In [20]:
Gdir.out_edges('l3139366') #### 4: 10680/ 36856

OutEdgeDataView([('l3139366', 'm4362177'), ('l3139366', 'x3139366a')])

In [21]:
Gdir.out_edges('l2785527') #5: 51201

OutEdgeDataView([('l2785527', 'l2766729'), ('l2785527', 'x2785527a')])

In [22]:
Gdir.out_edges('m1209817') #6: 31506

OutEdgeDataView([('m1209817', 'l2801909')])

In [23]:
Gdir.out_edges('m1209814') #7: 31505

OutEdgeDataView([('m1209814', 'm1209811'), ('m1209814', 'l2801909')])

In [24]:
Gdir.out_edges('m1209811') #8: 34563

OutEdgeDataView([('m1209811', 'm1209807')])

In [25]:
Gdir.out_edges('m1209797') #9: 44006

OutEdgeDataView([('m1209797', 'm1209800')])

In [26]:
Gdir.out_edges('m1209807') #10: 37223

OutEdgeDataView([('m1209807', 'm1209805')])

In [27]:
Gdir.out_edges('m1209748') #### 11: 30326/ 14510

OutEdgeDataView([('m1209748', 'l3142099')])

In [28]:
Gdir.out_edges('m1209795') #12: 1646

OutEdgeDataView([('m1209795', 'l2936268')])

In [29]:
Gdir.out_edges('m1186061') #### 13: 7747/ 13459

OutEdgeDataView([('m1186061', 'l2861223'), ('m1186061', 'p829977'), ('m1186061', 'l3139366')])

In [30]:
Gdir.out_edges('m1209753') #14: 42738

OutEdgeDataView([('m1209753', 'm1209748'), ('m1209753', 'm1209752')])

In [31]:
Gdir.out_edges('m1209763') #### 15: 17861/ 31529

OutEdgeDataView([('m1209763', 'l2936275')])

In [32]:
Gdir.out_edges('m1186078') #16: 15069

OutEdgeDataView([('m1186078', 'l2806553')])

In [33]:
Gdir.out_edges('m1209750') #### 17: 2222/ 38556

OutEdgeDataView([('m1209750', 'm1209751'), ('m1209750', 'l3086075')])

In [34]:
Gdir.out_edges('l2767340')

OutEdgeDataView([('l2767340', 'm3037449'), ('l2767340', 'm1125988'), ('l2767340', 'x2767340a'), ('l2767340', 'x2767340b'), ('l2767340', 'x2767340c')])

In [35]:
# nodes = ['m1209822', 'l2936216', 'l2823611', 'l3139366', 
#          'l2785527', 'm1209814', 'm1209811'
#          , 'm1209797', 'm1209807', 'm1209748',  'm1209795', 
#          'm1186061',  'm1209753', 'm1209763',  'm1186078', 
#          'm1209750']


In [36]:
nodes = ['m1209822', 'l2936216', 'l2823611', 'l3139366', 
         'l2785527', 'm1209817', 'm1209814', 'm1209811', 
         'm1209797', 'm1209807', 'm1209748',  'm1209795', 
         'm1186061',  'm1209753', 'm1209763',  'm1186078', 
         'm1209750']


In [37]:
#--- For an outage  scenario
OutLines = [('m1209822', 'm1209823'), ('m1209819', 'l2936216'), ('m1209787', 'l2823611'), ('l3139366', 'm4362177'), 
            ('l2785527', 'l2766729'), ('m1209817', 'l2801909'), ('m1209814', 'm1209811'), ('m1209811', 'm1209807'),
            ('m1209797', 'm1209800'),('m1209807', 'm1209805'), ('m1209748', 'l3142099'), ('m1209795', 'l2936268'), 
            ('m1186061', 'l2861223'), ('m1209753', 'm1209748'), ('m1209763', 'l2936275'), ('m1186078', 'l2806553'),
            ('m1209750', 'm1209751')] # This is to come from the reset in environment

# Initial graph state..with all lines damaged
Gout = G_primary.copy()

for oe in OutLines:
    Gout.remove_edge(*oe)

In [38]:
# Create the dictionary
node_outlines = {}
Power_restored_dict_old = {}

# Populate the dictionary
for node in nodes:
    node_outlines[node] = [outline for outline in OutLines if node in outline]

In [39]:
for node, outlines in node_outlines.items():
    repair_edge = outlines[0]
#     repair_edge = ('m1209822', 'm1209823') # this is the repaired edge coming from the agent
    (ru,rv) = repair_edge
    rlabel = G_primary.edges[ru,rv]['label']
    Gout.add_edge(ru, rv, label = rlabel)    
    restored_component, Power_restored = GraphRestore_Eval(Gout, substation_id) # This will give the power restored by the repair of the edges
    Power_restored_dict_old[node] = Power_restored

In [40]:
# Power_restored

In [41]:
Power_restored_dict_old

{'m1209822': 0,
 'l2936216': 0,
 'l2823611': 0,
 'l3139366': 0,
 'l2785527': 0,
 'm1209817': 4.07,
 'm1209814': 4.07,
 'm1209811': 4.07,
 'm1209797': 4.07,
 'm1209807': 4.07,
 'm1209748': 4.07,
 'm1209795': 4.07,
 'm1186061': 4.07,
 'm1209753': 4.07,
 'm1209763': 4.07,
 'm1186078': 4.07,
 'm1209750': 4.07}